<img src="https://github.com/jupytercon/2020-exactlyallan/raw/master/images/RAPIDS-header-graphic.png">

# Exploratory Data Visualization with Cuxfilter
***Quickly finding linked patterns in your data with cross filtering***


## Introduction

<img src="https://raw.githubusercontent.com/rapidsai/cuxfilter/main/docs/_images/demo.gif">

**Cuxfilter : "ku-cross-filter"** <br>
Part of **[RAPIDS](https://rapids.ai/)**, **[cuxfilter](https://github.com/rapidsai/cuxfilter)** is a library by the visualization team that enables GPU accelerated cross filtered dashboards in just a few lines of notebook code. You can find a list of features on our **[Docs page](https://docs.rapids.ai/api/cuxfilter/stable/)**.

Cuxfilter acts as a connector library rather than a visualization chart library. It abstracts away all the 'plumbing' required to inter-connect a **[curated list of visualizations](https://docs.rapids.ai/api/cuxfilter/stable/charts/charts.html)** to a cuDF GPU dataframe. By simply enabling accelerated dashboards within a notebook workflow, cuxfilter allows analysts to get to exploring their data faster.

**Part of a Visualization Workflow** <br>
A generalized python data visualization workflow invovles performing ETL with cuDF and charting libraries like **[hvplot](https://hvplot.holoviz.org/)** or **[holoviews](http://holoviews.org/)** (part of the great **[HoloViz](https://holoviz.org/)** eco-system), performing more detailed cross-filtered exploration and dashboard prototyping with cuxfilter, and then building stand alone explanatory data visualization appliactions with **[Plotly Dash](https://plotly.com/dash/)**.

Illustrating how easy it is to use RAPIDS libraries together, we will also use:

- **[cuDF](https://docs.rapids.ai/api/cudf/stable/)** a RAPIDS GPU DataFrame library for manipulating data with a pandas-like API.

- **[cuGraph](https://docs.rapids.ai/api/cugraph/stable/)** a RAPIDS GPU accelerated graph analytics library with functionality like NetworkX.

- **[cuSpatial](https://docs.rapids.ai/api/cusignal/stable/)** a RAPIDS GPU accelerated spatial analytics library used in cuxfilter's geospatial selections tools.

**Learn More** <br>
To hear more about RAPIDS, visualization, and additional resources, have a listen to our **[RAPIDSFire Podcast](https://anchor.fm/rapidsfire/episodes/Data-Visualization-at-Scale-with-Allan-Enemark-and-Bryan-Van-de-Ven-eq65l5)** or visit the **[RAPIDS Home Page](https://rapids.ai/)**.

A more extensive version of this tutorial can be found on our **[RAPIDS Community GitHub](https://github.com/rapidsai-community/showcase/tree/master/event_notebooks/JupyterCon_2020_RAPIDSViz)**.


## Installation and Requirements

For this tutorial you'll need an NVIDIA GPU with at least 16GB of memory and RAPIDS installed. You can find further requirements and installation instructions on our **[Getting Started Page](https://rapids.ai/start.html)**.

## Data Download
For this tutorial, we are using the **[Divvy Chicago bike share dataset](https://www.divvybikes.com/system-data)** sourced from this **[Kaggle page](https://www.kaggle.com/yingwurenjian/chicago-divvy-bicycle-sharing-data?select=data.csv)**.

In [1]:
from pathlib import Path
import cudf

DATA_DIR = Path("../data")
FILENAME = Path("data.csv")

In [2]:
# Download and Extract the dataset
! wget -N -P {DATA_DIR} https://rapidsai-data.s3.us-east-2.amazonaws.com/viz-data/data.tar.xz
! tar -xf {DATA_DIR}/data.tar.xz -C {DATA_DIR}

--2021-03-05 08:53:57--  https://rapidsai-data.s3.us-east-2.amazonaws.com/viz-data/data.tar.xz
Resolving rapidsai-data.s3.us-east-2.amazonaws.com (rapidsai-data.s3.us-east-2.amazonaws.com)... 52.219.105.74
Connecting to rapidsai-data.s3.us-east-2.amazonaws.com (rapidsai-data.s3.us-east-2.amazonaws.com)|52.219.105.74|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘../data/data.tar.xz’ not modified on server. Omitting download.



## Imports
Let's first make sure the necessary imports are present to load, as well as setting the data location.

In [3]:
import cuxfilter
import cudf
import cugraph
from bokeh.models import NumeralTickFormatter
from bokeh.palettes import Inferno
from pathlib import Path
from preprocess import * # for compactness we added functions to preprocess.py


## Load Data Into cuDF and Format Data
Load `datda.csv` into the GPU dataframe:

In [10]:
data = cudf.read_csv(DATA_DIR / FILENAME)

# Check
data

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,...,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,...,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,...,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495230,16734072,2017,10,39,6,0,Subscriber,Female,2017-10-01 00:01:00,2017-10-01 00:12:00,...,220,Hampden Ct & Diversey Pkwy,41.932620,-87.642385,23.0,87,Racine Ave & Fullerton Ave,41.925563,-87.658404,19.0
9495231,16734071,2017,10,39,6,0,Subscriber,Male,2017-10-01 00:01:00,2017-10-01 00:12:00,...,220,Hampden Ct & Diversey Pkwy,41.932620,-87.642385,23.0,87,Racine Ave & Fullerton Ave,41.925563,-87.658404,19.0
9495232,16734070,2017,10,39,6,0,Subscriber,Male,2017-10-01 00:01:00,2017-10-01 00:15:00,...,289,Wells St & Concord Ln,41.912133,-87.634656,19.0,226,Racine Ave & Belmont Ave,41.939743,-87.658865,15.0
9495233,16734067,2017,10,39,6,0,Subscriber,Female,2017-10-01 00:00:00,2017-10-01 00:06:00,...,288,Larrabee St & Armitage Ave,41.918084,-87.643749,11.0,289,Wells St & Concord Ln,41.912133,-87.634656,19.0


In [11]:
# Remove unnessary data and format using the script in preprocess.py
trips = process_trips(data)

# Check
trips.head()

,year,month,week,day,hour,gender,from_station_name,from_station_id,to_station_id,x,y,to_station_name,all_time_week,day_type
0,2014,6,27,0,23,Male,Lincoln Ave & Belmont Ave,131,303,-9.759200e+06,5.151901e+06,Broadway & Cornelia Ave,27,0
1,2014,6,27,0,23,Male,Halsted St & Maxwell St,282,22,-9.756812e+06,5.140716e+06,May St & Taylor St,27,0
2,2014,6,27,0,23,Male,Sheffield Ave & Webster Ave,327,225,-9.757567e+06,5.149256e+06,Halsted St & Dickens Ave,27,0
3,2014,6,27,0,23,Female,Peoria St & Jackson Blvd,134,194,-9.757113e+06,5.142684e+06,State St & Wacker Dr,27,0
4,2014,6,27,0,23,Female,Loomis St & Lexington St,320,134,-9.758434e+06,5.141853e+06,Peoria St & Jackson Blvd,27,0


In [6]:
# Create a weekday string map
days_of_week_map = {
    0: 'monday',
    1: 'tuesday',
    2: 'wednesday',
    3: 'thursday',
    4: 'friday',
    5: 'saturday',
    6: 'sunday'
}

# month map
month_map = {
    1: 'jan', 2: 'feb', 3: 'mar', 4: 'apr', 5: 'may', 6: 'jun', 7: 'jul', 8: 'aug', 9: 'sep', 10: 'oct', 11: 'nov', 12: 'dec'
}

# weekend / weekday map
day_type_map = {0:'weekday', 1:'weekend', '':'all'}

## An Aside  on Cognitive Load  
Creating the string maps above is important. To make the dashboard values more *human* understandable, we are creating string maps to convert the dataset's numbers to their proper names. Though it may seem trivial, it removes unnecessary ambiguity and helps **[reduce cognitive load](https://www.nngroup.com/articles/minimize-cognitive-load/)** when our focus needs to be on finding patterns.


## An Aside on Preattentive Attributes
Preattentive Attributes are a subconcious ability to quickly recognize patterns is due to our brain's natural ability to find **[preattentive attributes](http://daydreamingnumbers.com/blog/preattentive-attributes-example/)**, such as height, orientation, or color. Imagine 100 values in a table and 100 in a bar chart and how quickly you would be albe to find the smallest and largest values in either. This is one key reason visualizations are a powerful mechanism for recognizing patterns.

## Cuxfilter Basic Dashboard, Adding Charts, and Custom Layouts
First lets investigate trip totals by varous time slices by linking the dataframe to cuxfilter:

In [7]:
cux_df = cuxfilter.DataFrame.from_dataframe(data)

In [16]:
# Inferno Taken from bokeh color pallettes https://docs.bokeh.org/en/latest/docs/reference/palettes.html
colors = Inferno[10]

# Specify the charts and widgets to use with the selected columns of data and string maps
charts = [
    cuxfilter.charts.multi_select('year'),
    cuxfilter.charts.multi_select('day_type', label_map=day_type_map),
    cuxfilter.charts.bar('hour', title='trips per hour'),
    cuxfilter.charts.bar('month', x_label_map=month_map),
    cuxfilter.charts.bar('day', x_label_map=days_of_week_map)
]
# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")

# TRY:
# Add chart: cuxfilter.charts.datashader.heatmap(x='hour', y='day', aggregate_col='hour', point_shape='rect_horizontal', point_size=10, color_palette=colors)

# TRY:
# Use custom layout with `layout_array` parameter:
layout_array = [[1, 2], [3, 2]]

# Preset layout with `layout` parameter
layout = cuxfilter.layouts.feature_and_double_base

# Generate the dashboard and select a layout
d = cux_df.dashboard(charts, layout_array = layout_array, title='Bike Trips Dashboard')


In [17]:
# Start the dashboard, a green button should appear to open one in a new tab.
# NOTE: use the slider below each chart to cross filter.

# IMPORTANT: replace notebook_url with your jupyterhub/binder base url
# IMPORTANT: if your notebook environment is in jupyterhub, set service_proxy='jupyterhub', otherwise set to 'none'

BASE_URL = 'http://localhost:8888'
d.show(notebook_url=BASE_URL, service_proxy='none')

Dashboard running at port 59921


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

## Export Dashboard UI Queried Dataframe


In [15]:
# From last run querie on dashboard
queried_df = d.export()

# Check results
queried_df.columns

final query year==2014


Index(['trip_id', 'year', 'month', 'week', 'day', 'hour', 'usertype', 'gender',
       'starttime', 'stoptime', 'tripduration', 'temperature', 'events',
       'from_station_id', 'from_station_name', 'latitude_start',
       'longitude_start', 'dpcapacity_start', 'to_station_id',
       'to_station_name', 'latitude_end', 'longitude_end', 'dpcapacity_end',
       'x', 'y', 'day_type', 'all_time_week'],
      dtype='object')

## Cuxfilter Basic Dashboard, Preview, and Themes
Lets continue investigating, this time following up on the increasing trips year over year and decreases in winter months. 

In [23]:
# Specify the charts and widgets to use with the selected columns of data and string maps
charts = [
    cuxfilter.charts.bar('all_time_week', title='rides per week'),
    cuxfilter.charts.heatmap(x='all_time_week', y='day', aggregate_col='temperature',
                             aggregate_fn='mean', point_size=40, legend_position='right',
                             title='mean temperature by day'),
    cuxfilter.charts.multi_select('day_type', label_map=day_type_map),
]

# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")
        
# Generate the dashboard and select a layout
d = cux_df.dashboard(charts, 
                     layout=cuxfilter.layouts.feature_and_base, 
                     title='Temperature Dashboard', 
                     theme=cuxfilter.themes.rapids) #options: rapids, light, dark
# TRY: 
# Set different theme options and see the result in d.preview()


In [1]:
# Start the dashboard, a green button should appear to open one in a new tab.
# NOTE: pan to match up the top and bottom chart axis

# IMPORTANT: replace notebook_url with your jupyterhub/binder base url
# IMPORTANT: if your notebook environment is in jupyterhub, set service_proxy='jupyterhub', otherwise set to 'none'
d.show(notebook_url=BASE_URL, service_proxy='none')


# TRY:
# Use an inline image preview:
# await d.preview()


NameError: name 'd' is not defined

## Cuxfilter Geospatial Graph Dashboard
Next, lets take a look at the geospatial element of the data and see if we can find interesting patterns. Based on how the trip data is logged, converting it into a graph will make managing it easier.

For this we will need [cuGraph](https://docs.rapids.ai/api/cugraph/stable/api.html) to translate the dataset into an edge list:

In [2]:
# Build Graph
G = cugraph.Graph() 
G.from_cudf_edgelist(data, source='from_station_id', destination='to_station_id')
edges = G.edges()

NameError: name 'cugraph' is not defined

In [26]:
# Trips have been converted into edges with source and destination based on station IDs.
edges.head()

,src,dst
0,99,250
1,26,248
2,244,307
3,327,329
4,296,497


Next we load the formatted data into cuxfilter and specify the chart types:

In [27]:
cux_df = cuxfilter.DataFrame.load_graph((trips, edges))

In [28]:
# Specifying a graph chart type will use Datashader and its required parameters
charts = [
    cuxfilter.charts.multi_select('year'),
    cuxfilter.charts.multi_select('day_type', label_map=day_type_map),
    cuxfilter.charts.graph(
        node_id='from_station_id',
        edge_source='src', edge_target='dst',
        node_aggregate_fn='count',
        node_pixel_shade_type='linear', node_point_size=35, #node size is fixed set
        edge_render_type='direct', #direct, curved
        edge_transparency=0.7, #0.1 - 0.9
        tile_provider='CARTODBPOSITRON', 
        title='Graph for trip source_stations (color by count)'
    ),
    cuxfilter.charts.bar('from_station_id'),
    cuxfilter.charts.bar('to_station_id')
]

# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")

        
# Generate the dashboard, select a layout and theme
d = cux_df.dashboard(charts, layout=cuxfilter.layouts.feature_and_double_base, theme=cuxfilter.themes.rapids, title='Geospatial Trips')



In [29]:
# Start the dashboard, a green button should appear to open one in a new tab.
# NOTE: Graph edges can be turned on/off via the line tool icon
# NOTE: Inspect Neighboring Edges can be turned on/off for box or lasso select

# Caution: Selecting areas with `Inspect Neighboring Edges` on can result in slow performance or OOM errors  
# Caution: If the dashboard freezes, simply close the tab and restart this cell
# NOTE: This is rendering 9 MILLION edges!

# IMPORTANT: replace notebook_url with your jupyterhub/binder base url
# IMPORTANT: if your notebook environment is in jupyterhub, set service_proxy='jupyterhub', otherwise set to 'none'

d.show(notebook_url=BASE_URL, service_proxy='none')

Dashboard running at port 33421


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

## CuGraph Clustering
While the above produced many findings, filtering through so many trip edges is not ideal.
Next we will try to push the visual analytics further with a clustered network graph along side the geospatial graph using the [ForceAtlas2](https://docs.rapids.ai/api/cugraph/stable/api.html?highlight=force#module-cugraph.layout.force_atlas2) algorithm from cuGraph:

In [30]:
# NOTE: Often a good visualization result only comes from a lot of trial and error
# The below parameters produce useful clustering, but try experimenting with them further
ITERATIONS=500
THETA=10.0
OPTIMIZE=True

# Using the previously created edge list, we calculate the FA2 layout positions here
trips_force_atlas2_layout = cugraph.layout.force_atlas2(G, max_iter=ITERATIONS,
                strong_gravity_mode=False,
                outbound_attraction_distribution=True,
                lin_log_mode=False,
                barnes_hut_optimize=OPTIMIZE, barnes_hut_theta=THETA, verbose=True)

Merge the calculated forceAtlas2 layout with the trip dataframe:

In [31]:
final_df = trips_force_atlas2_layout.merge(
                trips[['from_station_id', 'from_station_name','to_station_id', 'year', 'hour', 'day_type', 'x', 'y']],
                left_on='vertex',
                right_on='from_station_id',
                suffixes=('', '_original')
)

# Check
final_df.head()

,x,y,vertex,from_station_id,from_station_name,to_station_id,year,hour,day_type,x_original,y_original
0,-1421.051758,3345.466064,173,173,Mies van der Rohe Way & Chicago Ave,334,2014,17,0,-9.754008e+06,5.145550e+06
1,-1355.209595,3420.928955,43,43,Michigan Ave & Washington St,174,2014,17,0,-9.754331e+06,5.143603e+06
2,-1655.240845,3472.884766,75,75,Canal St & Jackson Blvd,210,2014,17,0,-9.756126e+06,5.142720e+06
3,-1430.738403,3381.650146,100,100,Orleans St & Merchandise Mart Plaza,192,2014,17,0,-9.755638e+06,5.144253e+06
4,-1422.076416,3414.172852,98,98,LaSalle St & Washington St,321,2014,17,0,-9.755209e+06,5.143419e+06


## Cuxfilter Clustered Graph and Geospatial Dashboard Two

Next we load the data into cuxfilter and specify the chart types:

In [32]:
cux_df = cuxfilter.DataFrame.load_graph((final_df, edges))

In [33]:
# NOTE: Both scatter and graph chart types use Datashader 
charts= [
  cuxfilter.charts.graph(
      edge_source='src', edge_target='dst',
      edge_color_palette=['gray', 'black'],
      ode_pixel_shade_type='linear',
      edge_render_type='curved', #other option: direct
      edge_transparency=0.7, #0.1 - 0.9
      title='ForceAtlas2 Layout Graph'
  ),
  cuxfilter.charts.scatter(
    x='x_original', y='y_original', 
    tile_provider='CARTODBPOSITRON',
    point_size=3,
    pixel_shade_type='linear',
    pixel_spread='spread',
    title='Original Layout'
  ),
  cuxfilter.charts.bar('hour', title='Trips per hour'),
  cuxfilter.charts.bar('from_station_id', title='Source station'),
  cuxfilter.charts.bar('to_station_id', title='Destination station'),
  cuxfilter.charts.multi_select('year'),
  cuxfilter.charts.multi_select('day_type', label_map={0:'weekday', 1:'weekend', '':'all'})
] 

# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")

# Custom layout
layout_array_3rds = [[1,1,2],[1,1,2],[3,4,5]]

# Generate the dashboard, select a layout and theme
d = cux_df.dashboard(charts, layout_array = layout_array_3rds, theme=cuxfilter.themes.rapids, title="Network and Geospatial Graph")



In [34]:
# Start the dashboard, a green button should appear to open one in a new tab.
# NOTE: Graph edges can be turned on/off via the line tool icon
# NOTE: Inspect Neighboring Edges can be turned on/off for box or lasso select

# Caution: Selecting areas with Inspect Neighboring Edges on can result in slow performance or OOM errors  
# Caution: If the dashboard freezes, simply close the tab and restart this cell
# NOTE: This is rendering 9 MILLION edges

# IMPORTANT: replace notebook_url with your jupyterhub/binder base url
# IMPORTANT: if your notebook environment is in jupyterhub, set service_proxy='jupyterhub', otherwise set to 'none'

BASE_URL = 'http://localhost:8888'
d.show(notebook_url=BASE_URL, service_proxy='none')

Dashboard running at port 54185


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

## Visualization Findings

Running the FA2 algorithm to group the station nodes together in a graph and placing the geospatial chart along side provided some compelling findings:
- Stations form clusters of connectivity that are clearly geographically distinct 
- The core weekday group is actually multiple distinct clusters in close proximity (different work districts?)
- The weekday group stays focused until after work hours where they then disperse north (happy hour?)
- The weekend group is overall more spread out, starting along the coast then dispersing throughout the city towards the evening (sight seeing?)
- Theater on Lake Station is a hyper focal point for the weekend group

These are only a few notable points found relatively quickly - there are certainly more patterns.

As you can tell, by the end of this tutorial, finding patterns in the data is becoming the primary focus and the tooling is fading to the background. This is the behavior we hope to achieve with our 'interactive big data visuailzation' goal. 

## A Final Summary on the Benefits of Running with RAPIDS

Hopefully as you've clicked through this tutorial notebook, you've noticed how seamless it is working within the RAPIDS libraries and with other libraries. One of the key goals of RAPIDS is to keep the tools and workflows you are familiar with, but turn them into end-to-end GPU accelerated pipelines. From ETL, exploration, analytics, and visualization - you can take advantage of the speed ups from GPUs.

We on the viz team are continuing to integrate with other visualization libraries, and have projects in the works to improve the performance and capabilities of web visualizations even further.

## FYI: cuxfilter Troubleshooting
As we just released the graph visualization capability in cuxfilter, we are still working on building out features and fixes. 

If you find something that needs fixing or have feature requests, please submit an [issue on our Github Page](https://github.com/rapidsai/cuxfilter/issues). Better yet, [help contribute](https://github.com/rapidsai/cuxfilter#contributing-developers-guide). 